In [ ]:
import tensorflow as tf
from keras.layers import LSTM, GRU, Dense, Embedding, Dropout , Conv1D , GlobalMaxPool1D
from keras.preprocessing import text, sequence
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.utils.data_utils import pad_sequences
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import pandas as pd 
from sklearn import preprocessing
import re
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pickle

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [88]:
tokenizer = RegexpTokenizer(r'\w+')
 
nltk.download('stopwords')
stopwords_list = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ALI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [89]:
fake_isot_dataset = pd.read_csv("Fake.csv")
real_isot_dataset = pd.read_csv("True.csv")
fake_isot_dataset['Label'] =  0
real_isot_dataset['Label'] =  1
isot_dataset = pd.concat([fake_isot_dataset, real_isot_dataset], ignore_index=True)

# shuffle the dataset
isot_dataset = isot_dataset.sample(frac=1).reset_index(drop=True)
df = isot_dataset.copy()


In [90]:
isot_dataset.head()

,title,text,subject,date,Label
0,SHOCKING MUST WATCH VIDEO: Muslim Leaders Just...,Black Lives Matter is our campaign :,politics,"Mar 14, 2016",0
1,WATCH: Donald Trump Calls For Hillary Clinton...,Donald Trump told his supporters to engage in ...,News,"August 9, 2016",0
2,BREAKING UPDATE ON Obama’s War On Cops: Ohio P...,This random cop killing trend is almost like p...,politics,"Jan 18, 2016",0
3,German parties in coalition talks agree on no ...,BERLIN (Reuters) - Politicians from four Germa...,worldnews,"October 24, 2017",1
4,INVESTIGATION LAUNCHED: SECOND TRESPASSER May ...,The news that a second man was able to sneak i...,Government News,"Oct 27, 2017",0


In [91]:
isot_dataset["Title with text"] = isot_dataset["title"] + " " +  isot_dataset["text"]
isot_dataset = isot_dataset.to_numpy()
isot_y = isot_dataset[:, 4]
isot_X = isot_dataset[:, 5]

isot_X_postp = []
for i in range(isot_X.shape[0]):
    sentence = isot_X[i]
    sentence = re.sub(r'[^a-zA-Z ]', '', sentence)
    for stop_word in stopwords_list:
        sentence = sentence.replace(" " + stop_word + " ", " ")

    isot_X_postp.append(sentence)
isot_X = isot_X_postp 

In [92]:
vectorizer = TfidfVectorizer()
isot_X_tfidf = vectorizer.fit_transform(isot_X)
vectorizer = TfidfVectorizer(use_idf=False)
isot_X_tf = vectorizer.fit_transform(isot_X)

In [93]:
df.shape
df.head()

,title,text,subject,date,Label
0,SHOCKING MUST WATCH VIDEO: Muslim Leaders Just...,Black Lives Matter is our campaign :,politics,"Mar 14, 2016",0
1,WATCH: Donald Trump Calls For Hillary Clinton...,Donald Trump told his supporters to engage in ...,News,"August 9, 2016",0
2,BREAKING UPDATE ON Obama’s War On Cops: Ohio P...,This random cop killing trend is almost like p...,politics,"Jan 18, 2016",0
3,German parties in coalition talks agree on no ...,BERLIN (Reuters) - Politicians from four Germa...,worldnews,"October 24, 2017",1
4,INVESTIGATION LAUNCHED: SECOND TRESPASSER May ...,The news that a second man was able to sneak i...,Government News,"Oct 27, 2017",0


In [94]:
def clean_text(txt):
   
    contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
    def _get_contractions(contraction_dict):
        contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
        return contraction_dict, contraction_re

    def replace_contractions(text):
        contractions, contractions_re = _get_contractions(contraction_dict)
        def replace(match):
            return contractions[match.group(0)]
        return contractions_re.sub(replace, text)

    # replace contractions
    txt = replace_contractions(txt)
    
    #remove punctuations
    txt  = "".join([char for char in txt if char not in string.punctuation])
    txt = re.sub('[0-9]+', '', txt)
    
    # split into words
    words = word_tokenize(txt)
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    
    # removing leftover punctuations
    words = [word for word in words if word.isalpha()]
    
    cleaned_text = ' '.join(words)
    return cleaned_text
    
df['data_cleaned'] = df['title'].apply(lambda txt: clean_text(txt))


In [95]:
xtrain, xtest, ytrain, ytest = train_test_split(df['data_cleaned'], df['Label'], test_size=0.2 , shuffle=False)
isot_X_train, isot_X_test, isot_y_train, isot_y_test = train_test_split(isot_X_tfidf, isot_y, test_size=0.2 , shuffle=False)
isot_X_train_tf, isot_X_test_tf, isot_y_train_tf, isot_y_test_tf = train_test_split(isot_X_tf, isot_y, test_size=0.2 , shuffle=False)


In [96]:
# find the length of the largest sentence in training data
max_len = xtrain.apply(lambda x: len(x)).max()


In [97]:

max_words = 10000
tokenizer = text.Tokenizer(num_words = max_words)
# create the vocabulary by fitting on x_train text
tokenizer.fit_on_texts(xtrain)
# generate the sequence of tokens
xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xtest_seq = tokenizer.texts_to_sequences(xtest)

# pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xtest_pad = pad_sequences(xtest_seq, maxlen=max_len)
word_index = tokenizer.word_index

print('Text Example:', xtrain[0])
print('Sequence of indices(before padding):', xtrain_seq[0])
print('Sequence of indices(after padding):', xtrain_pad[0])

Text Example: SHOCKING MUST WATCH VIDEO Muslim Leaders Just Openly Declared War On The American
Sequence of indices(before padding): [348, 204, 18, 2, 71, 241, 28, 1928, 3680, 95, 10, 4, 164]
Sequence of indices(after padding): [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  

In [98]:
def eval_model(true_value, predict_value):
    print("accuracy:  ", accuracy_score(true_value, predict_value))
    print("Precision:  ", precision_score(true_value, predict_value))
    print("Recall:  ", recall_score(true_value, predict_value))
    print("F1 score:  ", f1_score(true_value, predict_value))


In [99]:
log_model = pickle.load(open("log_model_isot.pickle", "rb"))
predict_y_lr = log_model.predict(isot_X_train)


In [100]:
svm_clf = pickle.load(open("svm_clf_isot.pickle", "rb"))
predict_y_svm = svm_clf.predict(isot_X_train)


In [101]:
dt_clf = pickle.load(open("dt_clf_isot.pickle", "rb"))
predict_y_dt = dt_clf.predict(isot_X_train)


In [102]:
knn = pickle.load(open("knn_isot.pickle", "rb"))
predict_y_knn = knn.predict(isot_X_train)


In [103]:
rf1_clf = pickle.load(open("rf1_clf_isot.pickle", "rb"))
predict_y_rf1 = rf1_clf.predict(isot_X_train)


In [104]:
rf2_clf = pickle.load(open("rf2_clf_isot.pickle", "rb"))
predict_y_rf2 = rf2_clf.predict(isot_X_train_tf)


In [105]:
lstm_model = load_model('lstm_model_isot.h5')
predict_y_lstm = lstm_model.predict(xtrain_pad).round().astype(int).reshape(-1)


1123/1123 [==============================] - 114s 101ms/step


In [106]:
gru_model = load_model('gru_model_isot.h5')
predict_y_gru = gru_model.predict(xtrain_pad).round().astype(int).reshape(-1)


1123/1123 [==============================] - 61s 53ms/step


In [108]:
cnn_model = load_model('cnn_model_isot.h5')
predict_y_cnn = cnn_model.predict(xtrain_pad).round().astype(int).reshape(-1)


1123/1123 [==============================] - 8s 7ms/step


In [109]:
x_train_rf3 = []
y_train_rf3 = []
for i in range(len(xtrain)):
    x_train_rf3.append([predict_y_cnn[i] , predict_y_lstm[i] , predict_y_gru[i] , predict_y_dt[i] , predict_y_rf1[i] , predict_y_knn[i] , predict_y_lr[i] , predict_y_svm[i] , predict_y_rf2[i]])
    y_train_rf3.append(ytrain.iloc[i])


In [110]:
y_train_rf3 = np.array(y_train_rf3)


In [111]:
x_train_rf3

[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 1, 1, 1],
 [1, 0, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1,

In [113]:
y_train_rf3

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [114]:
rf3_model = RandomForestClassifier(n_estimators=1000, random_state=42)

rf3_model.fit(x_train_rf3, y_train_rf3)


RandomForestClassifier(n_estimators=1000, random_state=42)

In [115]:
# pickle.dump(rf3_model, file = open("rf3_model_isot.pickle", "wb"))


In [81]:
predict_y_rf3

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

### Test

In [117]:
predict_y_lr_test = log_model.predict(isot_X_test)



In [118]:
predict_y_svm_test = svm_clf.predict(isot_X_test)


In [119]:
predict_y_dt_test = dt_clf.predict(isot_X_test)

In [120]:
predict_y_knn_test = knn.predict(isot_X_test)

In [121]:
predict_y_cnn_test = cnn_model.predict(xtest_pad).round().astype(int).reshape(-1)

281/281 [==============================] - 2s 8ms/step


In [122]:
predict_y_gru_test = gru_model.predict(xtest_pad).round().astype(int).reshape(-1)

281/281 [==============================] - 14s 51ms/step


In [123]:
predict_y_lstm_test = lstm_model.predict(xtest_pad).round().astype(int).reshape(-1)

281/281 [==============================] - 38s 133ms/step


In [124]:
predict_y_rf2_test = rf2_clf.predict(isot_X_test_tf)

In [125]:
predict_y_rf1_test = rf1_clf.predict(isot_X_test)

In [126]:
x_test_rf3 = []
y_test_rf3 = []
for i in range(len(xtest)):
    x_test_rf3.append([predict_y_cnn_test[i] , predict_y_lstm_test[i] , predict_y_gru_test[i] , predict_y_dt_test[i] , predict_y_rf1_test[i] , predict_y_knn_test[i] , predict_y_lr_test[i] , predict_y_svm_test[i] , predict_y_rf2_test[i]])
    y_test_rf3.append(ytest.iloc[i])

y_test_rf3 = np.array(y_test_rf3)


In [128]:
rf3_model = pickle.load(open("rf3_model_isot.pickle", "rb"))
predict_y_rf3 = rf3_model.predict(x_test_rf3)

In [129]:
eval_model(y_test_rf3 , predict_y_rf3)

accuracy:   0.9994432071269488
Precision:   0.999073645206114
Recall:   0.9997682502896872
F1 score:   0.9994208270589598
